# Chapter 2 - Automated Testing

## Introducing the xUnit framework

- You can install the `C# Dev Kit` extension for VS Code to use the GUI for testing
    * You must build the test project for the tests to appear in the Test Explorer
- Creating a xUnit project `MyProject.Tests`


    ```bash
    dotnet new xunit
    ```

    Add a reference to the project you are testing.

    ```bash
    dotnet add reference ../../src/MyProject.csproj.
    ```

- `[fact]` attribute is used to define a test case
- `[Theroy]` attribute is used to define a data driven test case
- Running tests with the CLI

    ```
    dotnet test
    ```

In [2]:
#r "nuget:xunit"

Installed Packages xunit, 2.9.3

In [3]:
#load "SimpleTestRunner.cs"

In [ ]:
using Xunit;

public class CalculatorTests
{
    [Fact]
    public void Add_ShouldReturnCorrectSum()
    {
        int result = Add(2, 3);
        Assert.Equal(5, result);
    }

    [Fact]
    public void Add_TestShouldFail()
    {
        int result = Add(1, 1);
        Assert.Equal(3, result);
    }
    
    [Fact]
    public void TestException()
    {
        throw new Exception("This test is intentionally failing.");
    }

    private int Add(int a, int b) => a + b;
}

In [54]:
SimpleTestRunner.RunTests(typeof(CalculatorTests));

Running tests in CalculatorTests...
Running Add_ShouldReturnCorrectSum...
✔ Add_ShouldReturnCorrectSum passed.
Running Add_Failure...
✘ Add_Failure failed: Assert.Equal() Failure: Values differ
Expected: 3
Actual:   2
Running TestException...
✘ TestException failed: This test is intentionally failing.
   at Submission#51.CalculatorTests.TestException()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeWithNoArgs(Object obj, BindingFlags invokeAttr)

Use the [Theory] attribute to test with inline data.

In [55]:
using Xunit;

public class InlineDataTest
{
    [Theory]
    [InlineData(1, 1)]
    [InlineData(2, 2)]
    [InlineData(5, 5)]
    public void Should_be_equal(int value1, int value2)
    {
        Assert.Equal(value1, value2);
    }
}
SimpleTestRunner.RunTests(typeof(InlineDataTest));

Running tests in InlineDataTest...
Running Should_be_equal with arguments: 1,1...
✔ Should_be_equal passed.
Running Should_be_equal with arguments: 2,2...
✔ Should_be_equal passed.
Running Should_be_equal with arguments: 5,5...
✔ Should_be_equal passed.


## Fixture

- Inject dependencies into test classes
    * Good for costly dependencies like in-memory databases
- Setup - Use Constructor
- Teardown - Use `IDisposable`

## Arrange, Act, Assert

- Method writing readable tests
- Ideally Act will be a single line

In [4]:
using Xunit;

public class ArrangeActAssert
{
    [Fact]
    public void Should_be_equals()
    {
        // Arrange
        var a = 1;
        var b = 2;
        var expectedResult = 3;

        // Act
        var result = a + b;

        // Assert
        Assert.Equal(expectedResult, result);
    }
}

SimpleTestRunner.RunTests(typeof(ArrangeActAssert));


Running tests in ArrangeActAssert...
Running Should_be_equals...
✔ Should_be_equals passed.


## Organizing your tests

- Seperate projects for unit and integration tests
    * Recommendation only
    * Unit per project
    * 1 Integration per solution
- src directory for projects and test directory for test projects at the solution level
- Test use same namespace as code being tested
- Test directory structure matches the app being testsed. 

i.e.

```
src/MyApp/Controllers/HelloWorldController.cs
test/MApp.Tests/Controllers/HelloWorldControllerTest.cs
```

- Sub-class for each method being tested

In [6]:
#r "nuget:Microsoft.AspNetCore.Mvc"

Installed Packages Microsoft.AspNetCore.Mvc, 2.3.0

In [7]:
using Microsoft.AspNetCore.Mvc;

//namespace MyApp.Controllers;

[Route("")]
[ApiController]
public class HelloWorldController : ControllerBase
{
    [HttpGet]
    public string Hello()
    {
        return "Hello World!";
    }
}


In [8]:
using Xunit;

//namespace MyApp.Controllers;

public class HelloWorldControllerTest
{
    public class Hello : HelloWorldControllerTest
    {
        [Fact]
        public void Should_return_hello_world()
        {
            // Arrange
            var sut = new HelloWorldController();

            // Act
            var result = sut.Hello();

            // Assert
            Assert.Equal("Hello World!", result);
        }
    }
}
SimpleTestRunner.RunTests(typeof(ArrangeActAssert));

Running tests in ArrangeActAssert...
Running Should_be_equals...
✔ Should_be_equals passed.


## Integration Tests

- .NET Core added the `WebApplicationFactory<TEntry>` class to make integration testing easier in .NET

    ```
    dotnet add package Microsoft.ASPNetCore.Mvc.Testing
    ```
    
- Provides httpClient to query the web app
- Test classes have extension points to mock implementaitons
- Need a workaround to test the application - Add this to the bottom of `Program.cs`

    [C02/src/MyMinimalApiApp/Program.cs](C02/src/MyMinimalApiApp/Program.cs)
    
    ```csharp
    var builder = WebApplication.CreateBuilder(args);
    var app = builder.Build();
    app.MapGet("/", () => "Hello World!");
    app.Run();

    public partial class Program { }
    ```

    [C02/test/MyMinimalApiApp.IntegrationTests/ProgramTest.cs](C02/test/MyMinimalApiApp.IntegrationTests/ProgramTest.cs)

    ```csharp
    public class ProgramTest : IClassFixture<WebApplicationFactory<Program>>
    {
    private readonly HttpClient _httpClient;

    public ProgramTest(WebApplicationFactory<Program> webApplicationFactory)
    {
        _httpClient = webApplicationFactory.CreateClient();
    }

    public class Get : ProgramTest
    {
        public Get(WebApplicationFactory<Program> webApplicationFactory) : base(webApplicationFactory) { }

        [Fact]
        public async Task Should_respond_a_status_200_OK()
        {
            // Act
            var result = await _httpClient.GetAsync("/");

            // Assert
            Assert.Equal(HttpStatusCode.OK, result.StatusCode);
        }
    ```

> One essential thing to remember when writing tests is to test use cases, not the code itself; we are 
testing features’ correctness, not code correctness.

